In [1]:
import pandas as pd
import numpy as np
import os
from bokeh.io import output_notebook, show
output_notebook()
import bokeh as bh
from sklearn.preprocessing import LabelEncoder
import gc
from glob import glob
import re
from math import ceil
import xgboost as xgb
from sklearn.model_selection import GroupKFold,KFold,StratifiedKFold,train_test_split
import random
import operator
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import datetime 
import fastai.tabular as ft
pd.options.display.max_columns = 1000

Loading BokehJS ...

In [2]:
train_id = pd.read_csv('train_identity.csv')
train_trans = pd.read_csv('train_transaction.csv')
train_df = train_trans.merge(train_id,how='left',on=['TransactionID'])

In [3]:
test_id = pd.read_csv('test_identity.csv')
test_trans = pd.read_csv('test_transaction.csv')
test_df = test_trans.merge(test_id,how='left',on=['TransactionID'])

In [4]:
del train_id,train_trans,test_id,test_trans
gc.collect()

71

In [5]:
# prdm1 = pd.read_csv('predv1_data.csv')
# prdm1 = prdm1[prdm1['v1pred'].notnull()]
# prdm2 = pd.read_csv('predv1_data_v2ft.csv')
# prdm2 = prdm2[prdm2['v1pred'].notnull()]

In [6]:
# a = np.array(prdm1['v1pred'])
# b = np.array(prdm2['v1pred'])
# c = np.array(train_df['TransactionID'])
# pred = pd.DataFrame({'TransactionID':c, 'isFraudpred1':a,'isFraudpred2':b})

In [7]:
# t1 = pd.read_csv('fraud_preds_15xgb5f.csv')
# t2 = pd.read_csv('fraud_preds_15xgb5f2k_v2ft.csv')
# t3 = t1.merge(t2,how='left',on=['TransactionID'])
# t3.columns = ['TransactionID','isFraudpred1','isFraudpred2']

In [8]:
# train_df = train_df.merge(pred,how='left',on=['TransactionID'])
# test_df = test_df.merge(t3,how='left',on=['TransactionID'])


In [9]:

set(train_df['P_emaildomain'])

{'aim.com',
 'anonymous.com',
 'aol.com',
 'att.net',
 'bellsouth.net',
 'cableone.net',
 'centurylink.net',
 'cfl.rr.com',
 'charter.net',
 'comcast.net',
 'cox.net',
 'earthlink.net',
 'embarqmail.com',
 'frontier.com',
 'frontiernet.net',
 'gmail',
 'gmail.com',
 'gmx.de',
 'hotmail.co.uk',
 'hotmail.com',
 'hotmail.de',
 'hotmail.es',
 'hotmail.fr',
 'icloud.com',
 'juno.com',
 'live.com',
 'live.com.mx',
 'live.fr',
 'mac.com',
 'mail.com',
 'me.com',
 'msn.com',
 nan,
 'netzero.com',
 'netzero.net',
 'optonline.net',
 'outlook.com',
 'outlook.es',
 'prodigy.net.mx',
 'protonmail.com',
 'ptd.net',
 'q.com',
 'roadrunner.com',
 'rocketmail.com',
 'sbcglobal.net',
 'sc.rr.com',
 'servicios-ta.com',
 'suddenlink.net',
 'twc.com',
 'verizon.net',
 'web.de',
 'windstream.net',
 'yahoo.co.jp',
 'yahoo.co.uk',
 'yahoo.com',
 'yahoo.com.mx',
 'yahoo.de',
 'yahoo.es',
 'yahoo.fr',
 'ymail.com'}

In [10]:

emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum',
          'scranton.edu': 'other', 'netzero.net': 'other',
          'optonline.net': 'other', 'comcast.net': 'other', 
          'cfl.rr.com': 'other', 'sc.rr.com': 'other',
          'suddenlink.net': 'other', 'windstream.net': 'other',
          'gmx.de': 'other', 'earthlink.net': 'other', 
          'servicios-ta.com': 'other', 'bellsouth.net': 'other', 
          'web.de': 'other', 'mail.com': 'other',
          'cableone.net': 'other', 'roadrunner.com': 'other', 
          'protonmail.com': 'other', 'anonymous.com': 'other',
          'juno.com': 'other', 'ptd.net': 'other',
          'netzero.com': 'other', 'cox.net': 'other', 
          'hotmail.co.uk': 'microsoft', 
          'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 
          'live.com': 'microsoft', 'aim.com': 'aol',
          'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 
          'hotmail.com': 'microsoft',  
          'hotmail.fr': 'microsoft',
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 
          'yahoo.de': 'yahoo', 
          'live.fr': 'microsoft', 'verizon.net': 'yahoo', 
          'msn.com': 'microsoft', 'q.com': 'centurylink',
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 
           'rocketmail.com': 'yahoo', 
          'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 
          'embarqmail.com': 'centurylink', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo',
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft',
           'aol.com': 'aol', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

In [11]:
for c in ['P_emaildomain', 'R_emaildomain']:
    train_df[c + '_bin'] = train_df[c].map(emails)
    test_df[c + '_bin'] = test_df[c].map(emails)
    
    train_df[c + '_suffix'] = train_df[c].map(lambda x: str(x).split('.')[-1])
    test_df[c + '_suffix'] = test_df[c].map(lambda x: str(x).split('.')[-1])
    
    train_df[c + '_suffix'] = train_df[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test_df[c + '_suffix'] = test_df[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [12]:
def corret_card_id(x): 
    x=x.replace('.0','')
    x=x.replace('-999','nan')
    return x

def define_indexes(df):
    
    # create date column
    START_DATE = '2017-12-01'
    startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
    df['TransactionDT'] = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
    
    df['year'] = df['TransactionDT'].dt.year
    df['month'] = df['TransactionDT'].dt.month
    df['dow'] = df['TransactionDT'].dt.dayofweek
    df['hour'] = df['TransactionDT'].dt.hour
    df['day'] = df['TransactionDT'].dt.day
   
    # create card ID 
    cards_cols= ['card1', 'card2', 'card3', 'card5']
    for card in cards_cols: 
        if '1' in card: 
            df['card_id']= df[card].map(str)
        else : 
            df['card_id']+= ' '+df[card].map(str)
    
    # small correction of the Card_ID
    df['card_id']=df['card_id'].apply(corret_card_id)

    return df

In [13]:
train_df = define_indexes(train_df)
test_df = define_indexes(test_df)

In [14]:
train_df['Trans_min_mean'] = train_df['TransactionAmt'] - train_df['TransactionAmt'].mean()
train_df['Trans_min_std'] = train_df['Trans_min_mean'] / train_df['TransactionAmt'].std()
test_df['Trans_min_mean'] = test_df['TransactionAmt'] - test_df['TransactionAmt'].mean()
test_df['Trans_min_std'] = test_df['Trans_min_mean'] / test_df['TransactionAmt'].std()

In [15]:
train_df['TransactionAmt_to_mean_card1'] = train_df['TransactionAmt'] / train_df.groupby(['card1'])['TransactionAmt'].transform('mean')
train_df['TransactionAmt_to_mean_card4'] = train_df['TransactionAmt'] / train_df.groupby(['card4'])['TransactionAmt'].transform('mean')
train_df['TransactionAmt_to_std_card1'] = train_df['TransactionAmt'] / train_df.groupby(['card1'])['TransactionAmt'].transform('std')
train_df['TransactionAmt_to_std_card4'] = train_df['TransactionAmt'] / train_df.groupby(['card4'])['TransactionAmt'].transform('std')

test_df['TransactionAmt_to_mean_card1'] = test_df['TransactionAmt'] / test_df.groupby(['card1'])['TransactionAmt'].transform('mean')
test_df['TransactionAmt_to_mean_card4'] = test_df['TransactionAmt'] / test_df.groupby(['card4'])['TransactionAmt'].transform('mean')
test_df['TransactionAmt_to_std_card1'] = test_df['TransactionAmt'] / test_df.groupby(['card1'])['TransactionAmt'].transform('std')
test_df['TransactionAmt_to_std_card4'] = test_df['TransactionAmt'] / test_df.groupby(['card4'])['TransactionAmt'].transform('std')

In [16]:
train_df['first_value_card1'] = train_df['card1'].astype(str).str[0:1].astype(float)
train_df['two_value_card1'] = train_df['card1'].astype(str).str[0:2].astype(float)

test_df['first_value_card1'] = test_df['card1'].astype(str).str[0:1].astype(float)
test_df['two_value_card1'] = test_df['card1'].astype(str).str[0:2].astype(float)

train_df['card2'] = train_df['card2'].fillna(0)
train_df['first_value_card2'] = train_df['card2'].astype(str).str[0:1].astype(float)
train_df['two_value_card2'] = train_df['card2'].astype(str).str[0:2].astype(float)

test_df['card2'] = test_df['card2'].fillna(0)
test_df['first_value_card2'] = test_df['card2'].astype(str).str[0:1].astype(float)
test_df['two_value_card2'] = test_df['card2'].astype(str).str[0:2].astype(float)

In [17]:
cols = list(train_df.columns)
for i in ['isFraud','TransactionDT']:
    cols.remove(i)

In [18]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [19]:
train_mv = missing_data(train_df)
test_mv = missing_data(test_df)

In [20]:
col_exp_lst = []
for i in cols:
    if train_mv[i]['Percent'] > 90 and test_mv[i]['Percent'] >90:
        col_exp_lst.append(i)

In [21]:
for i in col_exp_lst:
    cols.remove(i)

In [22]:
train_df['TransactionAmt_decimal'] = ((train_df['TransactionAmt'] - train_df['TransactionAmt'].astype(int)) * 1000).astype(int)
test_df['TransactionAmt_decimal'] = ((test_df['TransactionAmt'] - test_df['TransactionAmt'].astype(int)) * 1000).astype(int)

# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
train_df['card1_count_full'] = train_df['card1'].map(pd.concat([train_df['card1'], test_df['card1']], ignore_index=True).value_counts(dropna=False))
test_df['card1_count_full'] = test_df['card1'].map(pd.concat([train_df['card1'], test_df['card1']], ignore_index=True).value_counts(dropna=False))

# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-featur

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train_df[feature] = train_df[f1].astype(str) + '_' + train_df[f2].astype(str)
    test_df[feature] = test_df[f1].astype(str) + '_' + test_df[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train_df[feature].astype(str).values) + list(test_df[feature].astype(str).values))
    train_df[feature] = le.transform(list(train_df[feature].astype(str).values))
    test_df[feature] = le.transform(list(test_df[feature].astype(str).values))
    
for feature in ['id_34', 'id_36']:
    if feature in cols:
        # Count encoded for both train and test
        train_df[feature + '_count_full'] = train_df[feature].map(pd.concat([train_df[feature], test_df[feature]], ignore_index=True).value_counts(dropna=False))
        test_df[feature + '_count_full'] = test_df[feature].map(pd.concat([train_df[feature], test_df[feature]], ignore_index=True).value_counts(dropna=False))
        
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
    if feature in cols:
        # Count encoded separately for train and test
        train_df[feature + '_count_dist'] = train_df[feature].map(train_df[feature].value_counts(dropna=False))
        test_df[feature + '_count_dist'] = test_df[feature].map(test_df[feature].value_counts(dropna=False))


In [23]:
cols_for_model = list(train_df.columns)
col_exp_lst.extend(['isFraud','TransactionDT'])
for i in col_exp_lst:
    print(i)
    cols_for_model.remove(i)

dist2
id_07
id_08
id_21
id_22
id_23
id_24
id_25
id_26
id_27
isFraud
TransactionDT


In [24]:
for f in cols_for_model:
    if train_df[f].dtype=='object':
        print(f+"-"+str(train_df[f].dtype))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values) + list(test_df[f].values))
        train_df[f] = lbl.transform(list(train_df[f].values))
        test_df[f] = lbl.transform(list(test_df[f].values))

ProductCD-object
card4-object
card6-object
P_emaildomain-object
R_emaildomain-object
M1-object
M2-object
M3-object
M4-object
M5-object
M6-object
M7-object
M8-object
M9-object
id_12-object
id_15-object
id_16-object
id_28-object
id_29-object
id_30-object
id_31-object
id_33-object
id_34-object
id_35-object
id_36-object
id_37-object
id_38-object
DeviceType-object
DeviceInfo-object
P_emaildomain_bin-object
P_emaildomain_suffix-object
R_emaildomain_bin-object
R_emaildomain_suffix-object
card_id-object


In [25]:
min(train_df['TransactionDT']) 

Timestamp('2017-12-02 00:00:00')

In [28]:
max(train_df['TransactionDT']) 

Timestamp('2018-06-01 23:58:51')

In [47]:
build_df = train_df[train_df['TransactionDT'] < datetime.datetime.strptime('2018-04-01', '%Y-%m-%d')].reset_index()
val_df = train_df[train_df['TransactionDT'] >= datetime.datetime.strptime('2018-04-01', '%Y-%m-%d')].reset_index()

In [48]:
class MetricsMeter():    
    def __init__(self, y_true, y_pred, threshold):
        self.y_true = y_true        
        self.y_pred = y_pred        
        self.thresh = threshold            
    
    def fit(self):        
        metrics = {}        
        metrics['auc'] = roc_auc_score(self.y_true, self.y_pred)                
        self.y_pred = [1 if x > self.thresh else 0 for x in self.y_pred]        
        metrics['f1'] = f1_score(self.y_true, self.y_pred,average = 'macro')        
        tn, fp, fn, tp = confusion_matrix(self.y_true, self.y_pred).ravel()                
        metrics['sensitivity/recall'] = tp / (tp+fn)
        metrics['precision'] = tp / (tp+fp)
        metrics["accuracy"] = (tp + tn) / (tp + fp + fn + tn)
        metrics['specificity'] = tn / (tn+fp)
        return metrics

In [49]:
def RunXGB(x_build,x_val,y_build,y_val,dtest,seed):
    dbuild = xgb.DMatrix(x_build,y_build)
    dval = xgb.DMatrix(x_val,y_val)
    watchlist = [(dbuild, 'train'), (dval, 'val')]
    params = {'objective': 'binary:logistic',
              'booster': 'gbtree',
              'eval_metric': 'auc',
              'nthread': 96,
              'max_depth': 7,
              #'learning_rate': 0.009
              'subsample': 0.8,
              'min_child_weight': 1,
              "colsample_bytree": 0.9,
              'eta': 0.08,
              'verbose_eval': True,
              'silent':1,
              'seed': seed
              }
    clf_xgb = xgb.train(params, dbuild, num_boost_round=1000, verbose_eval=50, early_stopping_rounds=30, evals=watchlist)
    pred_val = clf_xgb.predict(dval, ntree_limit=clf_xgb.best_iteration)
    pred_t = clf_xgb.predict(dtest, ntree_limit=clf_xgb.best_iteration)
    return(clf_xgb,pred_val,pred_t)

In [50]:
dtest = xgb.DMatrix(test_df[cols_for_model].iloc[:])
model1,pred_v1,pred_t1 = RunXGB(build_df[cols_for_model].iloc[:],val_df[cols_for_model].iloc[:],
build_df['isFraud'].iloc[:],val_df['isFraud'].iloc[:],dtest,seed = 9999)

[0]	train-auc:0.803663	val-auc:0.788866
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 30 rounds.
[50]	train-auc:0.919842	val-auc:0.880019
[100]	train-auc:0.952389	val-auc:0.903687
[150]	train-auc:0.964137	val-auc:0.909601
[200]	train-auc:0.970759	val-auc:0.91359
Stopping. Best iteration:
[200]	train-auc:0.970759	val-auc:0.91359



In [ ]:
train = train_df.reset_index(drop =True)
x_train = train[cols_for_model]
y_train = train['isFraud']
dtest = xgb.DMatrix(test_df[cols_for_model].iloc[:])
td = test_df['TransactionID'].reset_index(drop=True)

In [30]:
del train_df,test_df
gc.collect()

57

In [ ]:
# n_splits = 5
# cv_scores = []
# pred_test_f = 0
# pred_train = np.zeros(train.shape[0])
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=99)
# for build_index, val_index in kf.split(x_train, y_train):
#     x_build = x_train.iloc[build_index]
#     y_build = y_train.iloc[build_index]
#     x_val = x_train.iloc[val_index]
#     y_val = y_train.iloc[val_index]
#     pred_val = 0
#     pred_test = 0
#     n_models = 0.
    
#     model,pred_v,pred_t = RunXGB(x_build,x_val,y_build,y_val,dtest,seed = 10999973) 
#     pred_val += pred_v
#     pred_test += pred_t
#     n_models += 1
    
#     model,pred_v,pred_t = RunXGB(x_build,x_val,y_build,y_val,dtest,seed = 99) 
#     pred_val += pred_v
#     pred_test += pred_t
#     n_models += 1

    
#     model,pred_v,pred_t = RunXGB(x_build,x_val,y_build,y_val,dtest,seed = 90351) 
#     pred_val += pred_v
#     pred_test += pred_t
#     n_models += 1
    
#     pred_val /= n_models
#     pred_test /= n_models

    
#     pred_train[val_index] = pred_val
#     print(min(pred_train)),print(max(pred_train))
#     pred_test_f += pred_test / n_splits
#     print(min(pred_test_f)),print(max(pred_test_f))
#     #xgb_preds_lst.append(clf_xgb.predict(dtest))  
#     #models.append(clf_xgb)

In [ ]:
pred_df = pd.DataFrame(pred_t1 ,columns = ['isFraud'])

In [ ]:
fnl = pd.concat([td, pred_df], axis=1)

In [43]:
# predv1_data = pd.concat([train['TransactionID'],pd.DataFrame(pred_train ,columns = ['v1pred'])])

In [44]:
fnl.to_csv("fraud_preds_15xgb5f2k_tssp_vx3.csv",index=False)


In [116]:
predv1_data.to_csv("predv1_data_v2ft.csv",index=False)

In [ ]:
from google.colab import files

fnl.to_csv('fnl_part4.csv',index = False)
files.download('fnl_part4.csv')